<a href="https://colab.research.google.com/github/Lee-geun-bum/polars/blob/main/polars_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt 문제

In [ ]:
import os

print(os.getcwd())


/content


In [ ]:
df = pl.read_csv("tmp.csv")

df

user,task,trial,x,y,acc,valid
str,str,i64,i64,i64,i64,i64
"""U01""","""PS""",1,10,5,1,1
"""U01""","""PS""",2,8,4,1,1
"""U01""","""AS""",1,6,3,1,1
"""U01""","""AS""",2,6,2,0,1
"""U02""","""PS""",1,9,3,1,1
…,…,…,…,…,…,…
"""U03""","""AS""",2,8,4,1,1
"""U04""","""PS""",1,10,2,1,0
"""U04""","""PS""",2,5,5,1,1


In [ ]:
tmp = (
    df
    .filter(pl.col("valid")==1)
    .group_by(["user", "task"])
    .agg(
        pl.col("x").sum().alias("sum_x"),
        pl.col("y").sum().alias("sum_y"),
        pl.col("acc").mean().alias("acc_rate")
    )
    .with_columns(
        (pl.col("sum_x") /pl.col("sum_y")).alias("ratio_xy")
        )
    .filter(pl.col("acc_rate") >= 0.7)
    .group_by("task")
    .agg(
        pl.col("user").n_unique().alias("n_users"),
        pl.col("ratio_xy").mean().alias("mean_ratio"),
        pl.col("sum_x").sum().alias("total_sum_x")
    )
)

tmp

ColumnNotFoundError: task

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["user", "date", "item", "qty", ...]; PROJECT */7 COLUMNS

ans

In [ ]:

res = (
    df
    .filter(pl.col("valid") == 1)
    .group_by(["user", "task"])
    .agg(
        sum_x    = pl.col("x").sum(),
        sum_y    = pl.col("y").sum(),
        acc_rate = pl.col("acc").mean()
    )
    .with_columns(
        ratio_xy = pl.col("sum_x") / pl.col("sum_y")
    )
    .filter(pl.col("acc_rate") >= 0.7)
    .group_by("task")
    .agg(
        n_users     = pl.col("user").n_unique(),
        mean_ratio  = pl.col("ratio_xy").mean(),
        total_sum_x = pl.col("sum_x").sum()
    )
    .with_columns(pl.col("mean_ratio").round(4))
    .sort("task")
)

res


task,n_users,mean_ratio,total_sum_x
str,u32,f64,i64
"""AS""",2,1.5,21
"""PS""",3,1.6667,41


# 문제 2

In [ ]:
df = pl.read_csv("tmp.csv")
user = pl.read_csv("user.csv")

df


user,task,trial,x,y,acc,valid
str,str,i64,i64,i64,i64,i64
"""U01""","""PS""",1,10,5,1,1
"""U01""","""PS""",2,8,4,1,1
"""U01""","""AS""",1,6,3,1,1
"""U01""","""AS""",2,6,2,0,1
"""U02""","""PS""",1,9,3,1,1
…,…,…,…,…,…,…
"""U03""","""AS""",2,8,4,1,1
"""U04""","""PS""",1,10,2,1,0
"""U04""","""PS""",2,5,5,1,1


In [ ]:

res = (
    df
    .filter(pl.col("valid") == 1)
    .group_by(["user", "task"])
    .agg(
        sum_x    = pl.col("x").sum(),
        sum_y    = pl.col("y").sum(),
        acc_rate = pl.col("acc").mean()
    )
    .with_columns(
        ratio_xy = pl.col("sum_x") / pl.col("sum_y")
    )
    .filter(pl.col("acc_rate") >= 0.7)
    .join(user, on="user", how = "left")
    .group_by(["age", "task"])
    .agg(
        n_users     = pl.col("user").n_unique(),
        mean_ratio  = pl.col("ratio_xy").mean(),
        total_sum_x = pl.col("sum_x").sum()
    )
    .with_columns(pl.col("mean_ratio").round(4))
    .sort("task")
    .to_pandas()
)

res

,age,task,n_users,mean_ratio,total_sum_x
0,MCI,AS,1,2.0,16
1,CN,AS,1,1.0,5
2,CN,PS,2,2.0,36
3,MCI,PS,1,1.0,5


In [ ]:
## .to_pandas()만 하면 어짜피 나중에 뭐 호환성 문제는 고려안될듯

res_pd = res.to_pandas()
res_pd

,user,task,sum_x,sum_y,acc_rate,ratio_xy,age
0,U04,PS,5,5,1.0,1.0,MCI
1,U02,PS,18,9,1.0,2.0,CN
2,U02,AS,5,5,1.0,1.0,CN
3,U01,PS,18,9,1.0,2.0,CN
4,U03,AS,16,8,1.0,2.0,MCI


In [ ]:
import polars as pl

df = pl.read_csv("day2.1.csv")
user = pl.read_csv("day2.1-1.csv")

df

user,date,item,qty,price,acc,valid
str,str,str,i64,i64,i64,i64
"""U01""","""2025-01-01""","""A""",2,100,1,1
"""U01""","""2025-01-05""","""B""",1,200,1,1
"""U01""","""2025-01-10""","""C""",1,150,1,0
"""U02""","""2025-01-02""","""A""",3,100,1,1
"""U02""","""2025-01-03""","""B""",1,200,0,1
…,…,…,…,…,…,…
"""U03""","""2025-01-15""","""C""",2,150,1,1
"""U03""","""2025-01-25""","""B""",1,200,1,0
"""U04""","""2025-01-03""","""A""",1,100,1,1


In [ ]:
tmp = (
    df
    .filter(pl.col("valid") == 1)
    .with_columns(
        rev = pl.col("qty")*pl.col("price")
    )
    .group_by(["user", "item"])
    .agg(
        sum_qty = pl.col("qty").sum(),
        sum_rev = pl.col("rev").sum(),
        acc_item = pl.col("acc").mean()
    )
    .with_columns(
        acc_user = pl.col("acc_item").mean().over(pl.col("user"))
    )
    .filter(pl.col("acc_user") >= 0.8)
)

tmp = tmp.to_pandas()
tmp

,user,item,sum_qty,sum_rev,acc_item,acc_user
0,U01,B,1,200,1.0,1.0
1,U03,A,1,100,1.0,1.0
2,U03,C,2,300,1.0,1.0
3,U01,A,2,200,1.0,1.0


In [ ]:
tmp = (
    df
    .with_row_count(name="ID", offset=1)
    .filter(pl.col("valid") == 1)
    .with_columns(
        acc_user = pl.col("acc").mean().over(pl.col("user"))
    )
    .filter(pl.col("acc_user") >= 0.8)
    .with_columns(
        rev = pl.col("qty")*pl.col("price")
    )
    .group_by(["ID", "user", "item"])
    .agg(
        sum_qty = pl.col("qty").sum(),
        sum_rev = pl.col("rev").sum(),
        acc_item = pl.col("acc").mean()
    )
)


tmp = tmp.to_pandas()
tmp

/tmp/ipython-input-2289915840.py:3: DeprecationWarning: `DataFrame.with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count(name="ID", offset=1)


,ID,user,item,sum_qty,sum_rev,acc_item
0,2,U01,B,1,200,1.0
1,7,U03,A,1,100,1.0
2,8,U03,C,2,300,1.0
3,1,U01,A,2,200,1.0


NameError: name 'df' is not defined

# 꿀팁 - 기본

In [ ]:
## 인덱스
# 얘는 인덱스가 없어서 id를 만들고 하는게 도움될듯
# .with_row_count(name="ID", offset=1), offset = 시작순서


## str
# df.schema
# df.describe()


## nrow, ncol
# df.height
# df.width

## 첨에 체크하는 용도
# import pl.selector as cs

# df.select(
#     cs.all().max()         # min ~~
# )


## filter

# & | ~
# (pl.col("Type 1") == "Fire") & (pl.col("Attack") >= 100)
# ~(pl.col("Type 1").is_in(["Fire", "Water"]))
# pl.col("Type 2").is_null()
# pl.col("Type 2").is_not_null()
# pl.col("Name").str.contains("Mega")
# filter(pl.col("Name").str.contains("Mega"))
# filter(pl.col("Name").str.starts_with("D"))



## select

# select(pl.col("*").exclude(["Generation", "Legendary"]))
# import polars.selectors as cs  # 컬럼 선택을 위한 특별한 모듈

# # 모든 칼럼 선택
# selected_df = pl_df.select(pl.col("*"))
# selected_df = pl_df.select(pl.all())

# pl_df.select(pl.col(pl.Int64)) # 데이터 타입과 타입의 크기에 따라 선택 가능

# import polars.selectors as cs

# # int64, int32인지 구분하지 않고 정수형 칼럼을 선택하고 싶을 때
# pl_out_with_int = pl_df.select(cs.integer())

# # int64, int16, float32인지 구분하지 않고 실수형 칼럼과 문자열 칼럼을 선택하고 싶을 때
# pl_out_with_num = pl_df.select(cs.numeric(), cs.string())

# # 칼럼 이름에 정규식을 적용해서 선택(_encoded로 끝나는 칼럼)
# pl_df.select(pl.col("^.*_encoded$"))

# # 일부 열만 제외
# pl_df.select(pl.exclude("name", "private_number"))

# # 데이터 타입별 선택
# numeric_cols = df.select(cs.numeric())
# string_cols = df.select(cs.string())

# # 정규표현식을 사용한 선택
# type_cols = df.select(pl.col("^Type.*$"))  # "Type"으로 시작하는 모든 컬럼

# # 패턴 매칭을 통한 선택
# cols_with_type = df.select(cs.contains("Type"))  # "Type"이 포함된 컬럼들

# print(nueric_cols.head(1))
# print(string_cols.head(1))
# print(type_cols.head(1))
# print(cols_with_type.head(1))

# 요런 조건문도 됩니다요 ifelse
# select(
#     conditional = pl.when(("tmp")>2)
#                     .then(pl.lit(True))
#                     .otherwise(pl.lit(False))
# )


# with_columns 처럼 한번에 생성하면서 선택까지 가능한데,
# 쟤도 그렇고, select도 그렇고 그 안에서 만든건 못씀. 그 밑에 해야대
# ㅇㅇ 2차 가공 안됨



## sort

# 이런것도됨 앞에서부터 정렬
# sort(["Type 1", "Attack"], descending=[False, True])




## agg

# 리스트 안에 선택한 열들에 대해 다 평균때리고, _avg 붙여서 만든다 이거인듯
# .agg(
#         pl.col(["HP", "Attack", "Defense", "Speed"]).mean().name.suffix("_avg"),
#         pl.col(["HP", "Attack", "Defense", "Speed"]).std().name.suffix("_std"),
# )



# 꿀팁 - merge

In [2]:

## join

# df.join(moves_df, on="Name", how="left")

# 오 변수명 통일 안ㄴ시켜도 되는듯
# join(type_df, left_on="Type 1", right_on="Type", how="inner")



## cbind / rbind

# pl.concat(
#     [gen1, gen2], how = ("vertical" or "horizontal")
# )



## pivot

# .pivot(index="Pclass",on="Sex",values="Age",aggregate_function="mean")


## to_arrow()

# 벡터로 뽑을수있는데 뭔가 matrix 느낌인거 같기도하고
# df.select().to_arrow()
# 이런 느낌으로






In [ ]:
## 결측 확인

# polars는 NA 다 null로 처리

# | 목적          | pandas            | polars                              |

# | 결측 확인     | `df.isna()`       | `df.select(pl.all().is_null())`     |
# | 결측 아닌 값  | `df.notna()`      | `df.select(pl.all().is_not_null())` |
# | 결측 대체     | `df.fillna(0)`    | `df.fill_null(0)`                   |
# | 결측 제거     | `df.dropna()`     | `df.drop_nulls()`                   |
# | 결측 개수     | `df.isna().sum()` | `df.null_count()`                   |



## cast 타입변환

# df.with_columns(
#     pl.col("integer_col").cast(pl.Int32) # Int64를 Int32로 변환
# )

# "YYYY-MM-DD" 형식의 문자열을 Date 타입으로
# df.with_columns(
#     pl.col("date_string").cast(pl.Date, format="%Y-%m-%d")
# )


## 문자열

# strip = 양 끝 공백지우기, .. 소문자 변환
# with_columns(
#        pl.col("key").str.strip().str.to_lowercase()
# )
# 난중에 참고할 사이트
# https://zzinnam.com/polars-%EB%AC%B8%EC%9E%90%EC%97%B4-%EC%B2%98%EB%A6%AC-%EB%B0%A9%EB%B2%95-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0/



## 열 이름 변경

# 걍 r이랑 비슷
# df.columns = ["x1","x2","x3"]
# df.columns = [f"var_{i+1}" for i in range(len(df.columns))]





# 반복문 예제

In [ ]:
import polars as pl

In [ ]:
for i in range(1,5):
    print(i)

1
2
3
4


In [ ]:
import polars as pl

df_list = [pl.DataFrame({"x": [1, -2, 3]}), pl.DataFrame({"x": [-1, 5, 0]})]

for i in range(len(df_list)):
    df_list[i] = df_list[i].filter(pl.col("x") > 0)

df_list

[shape: (2, 1)
 ┌─────┐
 │ x   │
 │ --- │
 │ i64 │
 ╞═════╡
 │ 1   │
 │ 3   │
 └─────┘,
 shape: (1, 1)
 ┌─────┐
 │ x   │
 │ --- │
 │ i64 │
 ╞═════╡
 │ 5   │
 └─────┘]

In [ ]:
for idx, df in enumerate(df_list):
    df_list[idx] = df.with_columns((pl.col("x") ** 2).alias("x2"))

print(df_list)

[shape: (2, 2)
┌─────┬─────┐
│ x   ┆ x2  │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 1   ┆ 1   │
│ 3   ┆ 9   │
└─────┴─────┘, shape: (1, 2)
┌─────┬─────┐
│ x   ┆ x2  │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 5   ┆ 25  │
└─────┴─────┘]


In [ ]:
## 뜯어봐라 요고 .. 도움될듯
## 위아래

df = pl.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

df2 = df
df3 = df
df4 = df

for col in df.columns:
    mean_val = df.select(pl.col(col).mean())[0, 0]

    df = df.with_columns(
      pl.lit(mean_val).alias(f"{col}_mean")
    )


for col in df2.columns:
    cal_val = df2[col]*2 /10
    df2 = df2.with_columns(
      pl.lit(cal_val).alias(f"{col}_cal") # 이럴때는 alias로 반복문 안에서 편하게 변수생성
    )


df3 = df3.with_columns([
    pl.lit(float(df3[c].mean())).alias(f"{c}_mean") # 값이 1개면 lit으로 넣어야하는건가봐
    for c in df3.columns
])

df4 = df4.with_columns([
    (df4[c]*2/10).alias(f"{c}_mean")
    for c in df4.columns
])



print(df, df2, df3, df4)



shape: (3, 6)
┌─────┬─────┬─────┬────────┬────────┬────────┐
│ A   ┆ B   ┆ C   ┆ A_mean ┆ B_mean ┆ C_mean │
│ --- ┆ --- ┆ --- ┆ ---    ┆ ---    ┆ ---    │
│ i64 ┆ i64 ┆ i64 ┆ f64    ┆ f64    ┆ f64    │
╞═════╪═════╪═════╪════════╪════════╪════════╡
│ 1   ┆ 4   ┆ 7   ┆ 2.0    ┆ 5.0    ┆ 8.0    │
│ 2   ┆ 5   ┆ 8   ┆ 2.0    ┆ 5.0    ┆ 8.0    │
│ 3   ┆ 6   ┆ 9   ┆ 2.0    ┆ 5.0    ┆ 8.0    │
└─────┴─────┴─────┴────────┴────────┴────────┘ shape: (3, 6)
┌─────┬─────┬─────┬───────┬───────┬───────┐
│ A   ┆ B   ┆ C   ┆ A_cal ┆ B_cal ┆ C_cal │
│ --- ┆ --- ┆ --- ┆ ---   ┆ ---   ┆ ---   │
│ i64 ┆ i64 ┆ i64 ┆ f64   ┆ f64   ┆ f64   │
╞═════╪═════╪═════╪═══════╪═══════╪═══════╡
│ 1   ┆ 4   ┆ 7   ┆ 0.2   ┆ 0.8   ┆ 1.4   │
│ 2   ┆ 5   ┆ 8   ┆ 0.4   ┆ 1.0   ┆ 1.6   │
│ 3   ┆ 6   ┆ 9   ┆ 0.6   ┆ 1.2   ┆ 1.8   │
└─────┴─────┴─────┴───────┴───────┴───────┘ shape: (3, 6)
┌─────┬─────┬─────┬────────┬────────┬────────┐
│ A   ┆ B   ┆ C   ┆ A_mean ┆ B_mean ┆ C_mean │
│ --- ┆ --- ┆ --- ┆ ---    ┆ ---    ┆ ---    

In [ ]:
2

2

In [ ]:
import polars as pl

def process(df: pl.DataFrame) -> pl.DataFrame:
    # 예시: 결측 제거 → 파생열 추가 → 일부 열만 남기기
    return (
        df.drop_nulls()
          .with_columns(
              (pl.col("x") + pl.col("y")).alias("x_plus_y"),
              pl.col("x").mean().over([]).alias("x_mean")  # 상수열(전체 평균)
          )
          .select(["x", "y", "x_plus_y", "x_mean"])
    )


In [ ]:
dfs = [df1, df2, df3]

# A. 인덱스로 덮어쓰기 (enumerate)
for i, d in enumerate(dfs):
    dfs[i] = process(d)

# B. 한 줄로 새 리스트 만들기 (권장)
dfs = [process(d) for d in dfs]

In [ ]:
num = 55

print(f"I'm {num} year")

I'm 55 year


In [ ]:

tt = "3.141592 65358 979 323846 26433 83 279 502 884 1971 69 399 37510 5820 9749 44 592 30 78 164 062 862 0899 862 803 4825 342 117 0679 82"

# tt_s = pl.col("tt").str.strip()
# tt_s = str.strip(tt)

tt = tt.replace(" ","")

len(tt)


104